In [3]:
import librosa
import numpy as np
import os
import matplotlib.pyplot as plt

def preprocess_audio_folder(audio_folder_path, mel_output_folder_path, cqt_output_folder_path, sr=32000, n_fft=2048, hop_length=512, n_mels=128, batch_size=10):
    for root, dirs, files in os.walk(audio_folder_path):
        for dir in dirs:
            input_dir = os.path.join(root, dir)

            for file_name in os.listdir(input_dir):
                if file_name.endswith('.ogg'):
                    mel_output_dir = os.path.join(mel_output_folder_path, dir)
                    cqt_output_dir = os.path.join(cqt_output_folder_path, dir)
                    os.makedirs(mel_output_dir, exist_ok=True)
                    os.makedirs(cqt_output_dir, exist_ok=True)

                    # Load audio file
                    file_path = os.path.join(input_dir, file_name)
                    audio, _ = librosa.load(file_path, sr=sr)

                    # Reshape audio data if necessary
                    if len(audio.shape) > 1:
                        audio = np.mean(audio, axis=1)

                    # Compute Mel spectrogram
                    mel_spec = librosa.feature.melspectrogram(y=audio, sr=sr, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels)
                    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)

                    # Compute CQT spectrogram
                    cqt_spec = librosa.cqt(audio, sr=sr, hop_length=hop_length)
                    cqt_mag_spec = np.abs(cqt_spec)
                    cqt_mag_spec_db = librosa.amplitude_to_db(cqt_mag_spec, ref=np.max)

                    # Save Mel spectrogram as PNG image to Mel output folder
                    mel_output_file_path = os.path.join(mel_output_dir, file_name.replace('.ogg', '_mel.png'))
                    plt.figure(figsize=(10, 4))
                    librosa.display.specshow(mel_spec_db, sr=sr, hop_length=hop_length, x_axis='time', y_axis='mel')
                    plt.colorbar(format='%+2.0f dB')
                    plt.savefig(mel_output_file_path, bbox_inches='tight')
                    plt.close()

                    # Save CQT spectrogram as PNG image to CQT output folder
                    cqt_output_file_path = os.path.join(cqt_output_dir, file_name.replace('.ogg', '_cqt.png'))
                    plt.figure(figsize=(10, 4))
                    librosa.display.specshow(cqt_mag_spec_db, sr=sr, hop_length=hop_length, x_axis='time', y_axis='cqt_note')
                    plt.colorbar(format='%+2.0f dB')
                    plt.savefig(cqt_output_file_path, bbox_inches='tight')
                    plt.close()

                    # Clear memory
                    del audio, mel_spec, mel_spec_db, cqt_spec, cqt_mag_spec, cqt_mag_spec_db




In [4]:
preprocess_audio_folder('/Users/sammerking/Desktop/birds/train_audio', '/Users/sammerking/Desktop/birds/mel', '/Users/sammerking/Desktop/birds/cqt')


KeyboardInterrupt: 